# atmodeller

## Tutorial 1: basic operation

Import the required packages and set the package logger to the INFO level. For more output you could instead set it to DEBUG.

In [23]:
from atmodeller import InteriorAtmosphereSystem, Molecule, OCEAN_MOLES, MolarMasses, logger, FugacityConstraint, BufferedFugacityConstraint, SystemConstraint, MassConstraint
from atmodeller.thermodynamics import PeridotiteH2O, NoSolubility, BasaltDixonCO2, StandardGibbsFreeEnergyOfFormationHolland, StandardGibbsFreeEnergyOfFormationLinear

import logging

logger.setLevel(logging.DEBUG)

### 1. Simple H2O-H2 system with prescribed H2O pressure

We define a list of the molecules we wish to include in the interior-atmosphere system and how they partition between the melt and the atmosphere, and the solid and the melt:

In [24]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))

15:39:12 - atmodeller.core                - INFO      - Creating a molecule: H2O
15:39:12 - atmodeller.core                - DEBUG     - element count = 
{'H': 2, 'O': 1}
15:39:12 - atmodeller.core                - INFO      - Creating a molecule: H2
15:39:12 - atmodeller.core                - DEBUG     - element count = 
{'H': 2}


Although a choice is often made to constrain oxygen fugacity, we nevertheless need to explicitly include O2 as a molecule in the interior-atmosphere system:

In [25]:
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules

15:39:12 - atmodeller.core                - INFO      - Creating a molecule: O2
15:39:12 - atmodeller.core                - DEBUG     - element count = 
{'O': 2}


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x127db2490>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, molar_mass=0.018015200000000002),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x127d7f410>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, molar_mass=0.0020158),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x127da4790>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, molar_mass=0.0319988)]

We can then create an interior-atmosphere system using the list of molecules and (optionally) specifying the thermodynamic data to use. Note that this creates a planet with 'default properties' (a molten Earth). Adjusting the planet properties will be covered in a later tutorial.

In [26]:
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules, gibbs_data=StandardGibbsFreeEnergyOfFormationLinear())

15:39:12 - atmodeller.core                - INFO      - Creating a new planet
15:39:12 - atmodeller.core                - INFO      - Mantle mass (kg) = 4.208261222595111e+24
15:39:12 - atmodeller.core                - INFO      - Mantle melt fraction = 1.0
15:39:12 - atmodeller.core                - INFO      - Core mass fraction = 0.295334691460966
15:39:12 - atmodeller.core                - INFO      - Planetary radius (m) = 6371000.0
15:39:12 - atmodeller.core                - INFO      - Planetary mass (kg) = 5.972e+24
15:39:12 - atmodeller.core                - INFO      - Surface temperature (K) = 2000.000000
15:39:12 - atmodeller.core                - INFO      - Surface gravity (m/s^2) = 9.819973426224687
15:39:12 - atmodeller.core                - INFO      - Melt Composition = None
15:39:12 - atmodeller.core                - INFO      - Creating a new interior-atmosphere system
15:39:12 - atmodeller.core                - INFO      - Molecules = ['H2', 'O2', 'H2O']
15:39:12 -

Notice that for this simple system it has identified the single reaction that relates the 3 chosen molecules. To solve the system, we provide a constraint of the H2O pressure in bar:

In [27]:
H2O_pressure: SystemConstraint = FugacityConstraint(species='H2O', value=1)
constraints: list = [H2O_pressure]

A second constraint is required to close the system of equations, and often this is constraining the fugacity of some species at a buffered value, such as constraining the oxygen fugacity at the iron-wustite (IW) buffer. Constraining the fO2 at the IW buffer is the default if no arguments are provided:

In [28]:
O2_pressure: SystemConstraint = BufferedFugacityConstraint()
constraints.append(O2_pressure)
logger.info(constraints)

15:39:13 - atmodeller                     - INFO      - [FugacityConstraint(species='H2O', value=1, field='fugacity'), BufferedFugacityConstraint(species='O2', fugacity=<atmodeller.thermodynamics.IronWustiteBufferOneill object at 0x127de6990>, log10_shift=0, field='fugacity')]


In [29]:
interior_atmosphere.solve(constraints)

15:39:13 - atmodeller.core                - INFO      - Constraints: [FugacityConstraint(species='H2O', value=1, field='fugacity'),
 BufferedFugacityConstraint(species='O2',
                            fugacity=<atmodeller.thermodynamics.IronWustiteBufferOneill object at 0x127de6990>,
                            log10_shift=0,
                            field='fugacity')]
15:39:13 - atmodeller.core                - INFO      - Mixed pressure and mass constraints so attempting to solve a non-linear system of equations
15:39:13 - atmodeller.core                - INFO      - The necessary number of constraints will be applied to the reaction network to solve the system
15:39:13 - atmodeller.core                - INFO      - Row 00: Reaction 0: 1.0 H2 + 0.5 O2 = 1.0 H2O
15:39:13 - atmodeller.thermodynamics      - DEBUG     - Molecule = H2, standard Gibbs energy of formation = 0.000000
15:39:13 - atmodeller.thermodynamics      - DEBUG     - Molecule = O2, standard Gibbs energy of formation

{'H2': 1.4662542407051478, 'O2': 3.910416694617766e-08, 'H2O': 1.0}

You can access the solution directly using:

In [30]:
interior_atmosphere.fugacities_dict

{'H2': 1.4662542407051478, 'O2': 3.910416694617766e-08, 'H2O': 1.0}

### 2. System with C and H and prescribed pressures

We now extend the molecule list to additionally include C-species:

In [31]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
molecules

15:39:13 - atmodeller.core                - INFO      - Creating a molecule: H2O
15:39:13 - atmodeller.core                - DEBUG     - element count = 
{'H': 2, 'O': 1}
15:39:13 - atmodeller.core                - INFO      - Creating a molecule: H2
15:39:13 - atmodeller.core                - DEBUG     - element count = 
{'H': 2}
15:39:13 - atmodeller.core                - INFO      - Creating a molecule: O2
15:39:13 - atmodeller.core                - DEBUG     - element count = 
{'O': 2}
15:39:13 - atmodeller.core                - INFO      - Creating a molecule: CO
15:39:13 - atmodeller.core                - DEBUG     - element count = 
{'C': 1, 'O': 1}
15:39:13 - atmodeller.core                - INFO      - Creating a molecule: CO2
15:39:13 - atmodeller.core                - DEBUG     - element count = 
{'C': 1, 'O': 2}


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x127691310>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, molar_mass=0.018015200000000002),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x127d86950>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, molar_mass=0.0020158),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x127d7fa50>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, molar_mass=0.0319988),
 Molecule(name='CO', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x127da4d90>, solid_melt_distribution_coefficient=0, elements={'C': 1, 'O': 1}, element_masses={'C': 0.0120107, 'O': 0.0159994}, molar_mass=0.0280101),
 Molecule(name='CO2', solubility=<atmodeller.thermodynamics.BasaltDixonCO2 object at 0x11d846590>, 

In [32]:
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules)

15:39:13 - atmodeller.core                - INFO      - Creating a new planet
15:39:13 - atmodeller.core                - INFO      - Mantle mass (kg) = 4.208261222595111e+24
15:39:13 - atmodeller.core                - INFO      - Mantle melt fraction = 1.0
15:39:13 - atmodeller.core                - INFO      - Core mass fraction = 0.295334691460966
15:39:13 - atmodeller.core                - INFO      - Planetary radius (m) = 6371000.0
15:39:13 - atmodeller.core                - INFO      - Planetary mass (kg) = 5.972e+24
15:39:13 - atmodeller.core                - INFO      - Surface temperature (K) = 2000.000000
15:39:13 - atmodeller.core                - INFO      - Surface gravity (m/s^2) = 9.819973426224687
15:39:13 - atmodeller.core                - INFO      - Melt Composition = None
15:39:13 - atmodeller.core                - INFO      - Creating a new interior-atmosphere system
15:39:13 - atmodeller.core                - INFO      - Molecules = ['CO', 'H2', 'O2', 'CO2', 'H2O

Note now the system has identified two reactions in the network. With C present in the system we must provide at least 2 constraints, in addition to the oxygen fugacity:

In [33]:
H2O_pressure: SystemConstraint = FugacityConstraint(species='H2O', value=1)
CO2_pressure: SystemConstraint = FugacityConstraint(species='CO2', value=1)
O2_pressure: SystemConstraint = BufferedFugacityConstraint()
constraints: list[SystemConstraint] = [H2O_pressure, CO2_pressure, O2_pressure]

In [34]:
interior_atmosphere.solve(constraints)
interior_atmosphere.fugacities_dict

15:39:13 - atmodeller.core                - INFO      - Constraints: [FugacityConstraint(species='H2O', value=1, field='fugacity'),
 FugacityConstraint(species='CO2', value=1, field='fugacity'),
 BufferedFugacityConstraint(species='O2',
                            fugacity=<atmodeller.thermodynamics.IronWustiteBufferOneill object at 0x1276b9210>,
                            log10_shift=0,
                            field='fugacity')]
15:39:13 - atmodeller.core                - INFO      - Mixed pressure and mass constraints so attempting to solve a non-linear system of equations
15:39:13 - atmodeller.core                - INFO      - The necessary number of constraints will be applied to the reaction network to solve the system
15:39:13 - atmodeller.core                - INFO      - Row 00: Reaction 0: 1.0 CO + 0.5 O2 = 1.0 CO2
15:39:13 - atmodeller.thermodynamics      - DEBUG     - Molecule = CO, standard Gibbs energy of formation = -285748.794118
15:39:13 - atmodeller.thermodynamics

{'CO': 6.580633105623207,
 'H2': 1.4662542407051478,
 'O2': 3.910416694617766e-08,
 'CO2': 1.0,
 'H2O': 1.0}

There is not a requirement to necessarily impose the oxygen fugacity as a constraint. Instead, we can simply impose three pressure constraints (that span the reaction set) and allow for the oxygen fugacity to be solved. Note that if we do not specify an appropriate range of constraints we cannot solve the system of equations to give a unique solution and hence the code will raise an exception relating to a singular matrix.

In [35]:
H2O_pressure: SystemConstraint = FugacityConstraint(species='H2O', value=1)
H2_pressure: SystemConstraint = FugacityConstraint(species='H2', value=1)
CO_pressure: SystemConstraint = FugacityConstraint(species='CO', value=1)
constraints: list[SystemConstraint] = [H2O_pressure, H2_pressure, CO_pressure]

In [36]:
interior_atmosphere.solve(constraints)
interior_atmosphere.fugacities_dict

15:39:13 - atmodeller.core                - INFO      - Constraints: [FugacityConstraint(species='H2O', value=1, field='fugacity'),
 FugacityConstraint(species='H2', value=1, field='fugacity'),
 FugacityConstraint(species='CO', value=1, field='fugacity')]
15:39:13 - atmodeller.core                - INFO      - Mixed pressure and mass constraints so attempting to solve a non-linear system of equations
15:39:13 - atmodeller.core                - INFO      - The necessary number of constraints will be applied to the reaction network to solve the system
15:39:13 - atmodeller.core                - INFO      - Row 00: Reaction 0: 1.0 CO + 0.5 O2 = 1.0 CO2
15:39:13 - atmodeller.thermodynamics      - DEBUG     - Molecule = CO, standard Gibbs energy of formation = -285748.794118
15:39:13 - atmodeller.thermodynamics      - DEBUG     - Molecule = H2, standard Gibbs energy of formation = 0.000000
15:39:13 - atmodeller.thermodynamics      - DEBUG     - Molecule = O2, standard Gibbs energy of format

{'CO': 1.0,
 'H2': 1.0,
 'O2': 8.407010711071694e-08,
 'CO2': 0.22281355261277522,
 'H2O': 1.0}

### 3. System with C and H and mixed constraints

A typical use case is to define an interior-atmosphere system with a combination of pressure and mass constraints. We define the same molecule set as before:

In [37]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
molecules

15:39:13 - atmodeller.core                - INFO      - Creating a molecule: H2O
15:39:13 - atmodeller.core                - DEBUG     - element count = 
{'H': 2, 'O': 1}
15:39:13 - atmodeller.core                - INFO      - Creating a molecule: H2
15:39:13 - atmodeller.core                - DEBUG     - element count = 
{'H': 2}
15:39:13 - atmodeller.core                - INFO      - Creating a molecule: O2
15:39:13 - atmodeller.core                - DEBUG     - element count = 
{'O': 2}
15:39:13 - atmodeller.core                - INFO      - Creating a molecule: CO
15:39:13 - atmodeller.core                - DEBUG     - element count = 
{'C': 1, 'O': 1}
15:39:13 - atmodeller.core                - INFO      - Creating a molecule: CO2
15:39:13 - atmodeller.core                - DEBUG     - element count = 
{'C': 1, 'O': 2}


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x127d85750>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, molar_mass=0.018015200000000002),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x127db0f10>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, molar_mass=0.0020158),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x127d81e10>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, molar_mass=0.0319988),
 Molecule(name='CO', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x127da4490>, solid_melt_distribution_coefficient=0, elements={'C': 1, 'O': 1}, element_masses={'C': 0.0120107, 'O': 0.0159994}, molar_mass=0.0280101),
 Molecule(name='CO2', solubility=<atmodeller.thermodynamics.BasaltDixonCO2 object at 0x127d86bd0>, 

Now we define the constraints, and in this case we want to constrain the total mass of C and H in the system that can partition between the various reservoirs.

In [38]:
number_of_earth_oceans: float = 1
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = number_of_earth_oceans * OCEAN_MOLES * MolarMasses().H2
mass_C: float = ch_ratio * mass_H

constraints: list[SystemConstraint] = [
    MassConstraint(species="H", value=mass_H),
    MassConstraint(species="C", value=mass_C),
    BufferedFugacityConstraint()
]

interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules)
interior_atmosphere.solve(constraints)
interior_atmosphere.fugacities_dict

15:39:13 - atmodeller.core                - INFO      - Creating a new planet
15:39:13 - atmodeller.core                - INFO      - Mantle mass (kg) = 4.208261222595111e+24
15:39:13 - atmodeller.core                - INFO      - Mantle melt fraction = 1.0
15:39:13 - atmodeller.core                - INFO      - Core mass fraction = 0.295334691460966
15:39:13 - atmodeller.core                - INFO      - Planetary radius (m) = 6371000.0
15:39:13 - atmodeller.core                - INFO      - Planetary mass (kg) = 5.972e+24
15:39:13 - atmodeller.core                - INFO      - Surface temperature (K) = 2000.000000
15:39:13 - atmodeller.core                - INFO      - Surface gravity (m/s^2) = 9.819973426224687
15:39:13 - atmodeller.core                - INFO      - Melt Composition = None
15:39:13 - atmodeller.core                - INFO      - Creating a new interior-atmosphere system
15:39:13 - atmodeller.core                - INFO      - Molecules = ['CO', 'H2', 'O2', 'CO2', 'H2O

{'CO': 62.28167551155309,
 'H2': 0.5760309184494741,
 'O2': 3.910416694617766e-08,
 'CO2': 9.464389597762683,
 'H2O': 0.39285882520104465}

### 4. Including more species such as CH4

It is straightforward to add more species to the system, although they must have their formation energies and masses already specified in the code.

In [39]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CH4', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules

15:39:13 - atmodeller.core                - INFO      - Creating a molecule: H2O
15:39:13 - atmodeller.core                - DEBUG     - element count = 
{'H': 2, 'O': 1}
15:39:13 - atmodeller.core                - INFO      - Creating a molecule: H2
15:39:13 - atmodeller.core                - DEBUG     - element count = 
{'H': 2}
15:39:13 - atmodeller.core                - INFO      - Creating a molecule: O2
15:39:13 - atmodeller.core                - DEBUG     - element count = 
{'O': 2}
15:39:13 - atmodeller.core                - INFO      - Creating a molecule: CO
15:39:13 - atmodeller.core                - DEBUG     - element count = 
{'C': 1, 'O': 1}
15:39:13 - atmodeller.core                - INFO      - Creating a molecule: CO2
15:39:13 - atmodeller.core                - DEBUG     - element count = 
{'C': 1, 'O': 2}
15:39:13 - atmodeller.core                - INFO      - Creating a molecule: CH4
15:39:13 - atmodeller.core                - DEBUG     - element count = 
{'C': 1, '

[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x12769d050>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, molar_mass=0.018015200000000002),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x12769e650>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, molar_mass=0.0020158),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x127d79010>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, molar_mass=0.0319988),
 Molecule(name='CO', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x11ea63d90>, solid_melt_distribution_coefficient=0, elements={'C': 1, 'O': 1}, element_masses={'C': 0.0120107, 'O': 0.0159994}, molar_mass=0.0280101),
 Molecule(name='CO2', solubility=<atmodeller.thermodynamics.BasaltDixonCO2 object at 0x1276b9f90>, 

We define a mixture of mass and oxygen fugacity constraints as before and solve the system. CH4 is not prevalent at 2000 K so the results are almost identical to those without CH4 presented above.

In [40]:
number_of_earth_oceans: float = 1
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = number_of_earth_oceans * OCEAN_MOLES * MolarMasses().H2
mass_C: float = ch_ratio * mass_H

constraints: list[SystemConstraint] = [
    MassConstraint(species="H", value=mass_H),
    MassConstraint(species="C", value=mass_C),
    BufferedFugacityConstraint()
]

interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules)
interior_atmosphere.solve(constraints)
interior_atmosphere.fugacities_dict

15:39:13 - atmodeller.core                - INFO      - Creating a new planet
15:39:13 - atmodeller.core                - INFO      - Mantle mass (kg) = 4.208261222595111e+24
15:39:13 - atmodeller.core                - INFO      - Mantle melt fraction = 1.0
15:39:13 - atmodeller.core                - INFO      - Core mass fraction = 0.295334691460966
15:39:13 - atmodeller.core                - INFO      - Planetary radius (m) = 6371000.0
15:39:13 - atmodeller.core                - INFO      - Planetary mass (kg) = 5.972e+24
15:39:13 - atmodeller.core                - INFO      - Surface temperature (K) = 2000.000000
15:39:13 - atmodeller.core                - INFO      - Surface gravity (m/s^2) = 9.819973426224687
15:39:13 - atmodeller.core                - INFO      - Melt Composition = None
15:39:13 - atmodeller.core                - INFO      - Creating a new interior-atmosphere system
15:39:13 - atmodeller.core                - INFO      - Molecules = ['CO', 'H2', 'O2', 'CO2', 'H2O

{'CO': 62.28167381126504,
 'H2': 0.576030911226698,
 'O2': 3.910416694617766e-08,
 'CO2': 9.464389339385107,
 'H2O': 0.39285882027503966,
 'CH4': 1.3829358485053366e-06}